# PopJSON + R tutorial: Larval population with temperature-dependent Erlang timers

This notebook demonstrates how to:
1. Construct a PopJSON model for a larval stage with **temperature-dependent Erlang-distributed lifetime** and **temperature-dependent Erlang-distributed development time**.
2. Simulate an **ODE analogue** that represents these two Erlang 'clocks' as **interacting linear chains** (a phase-type / CTMC construction).
3. Compare outputs (larval abundance, cumulative pupation, cumulative mortality) under a varying temperature.

References:  
- PopJSON docs: https://kerguler.github.io/PopJSON/  
- sPop / accumulative-process formulation: Erguler et al. 2022; Erguler 2020.


## 1) Setup

In [ ]:
# install.packages(c('jsonlite','deSolve','ggplot2','dplyr','tidyr'))
library(jsonlite)
library(deSolve)
library(ggplot2)
library(dplyr)
library(tidyr)


## 2) Temperature forcing and process mappings

In [ ]:
T_seasonal <- function(t, T_mean = 24, amplitude = 6, phase = 30, period = 365) {
  T_mean + amplitude * sin(2*pi*(t - phase)/period)
}

briere1_recip <- function(T, a, L, R) {
  val <- ifelse(T <= L | T >= R, 365.0,
                1.0 / exp(a + log(pmax(T,1e-9)) + log(pmax(T-L,1e-9)) + 0.5*log(pmax(R-T,1e-9))))
  pmin(365.0, pmax(1.0, val))
}

dev_mean_days <- function(T) briere1_recip(T, a=-15, L=10, R=35)
life_mean_days <- function(T) briere1_recip(T, a=-14, L=8, R=38)

k_dev <- 8; k_life <- 4
dev_sd_days <- function(T) dev_mean_days(T)/sqrt(k_dev)
life_sd_days <- function(T) life_mean_days(T)/sqrt(k_life)

rate_dev <- function(t) { k_dev / dev_mean_days(T_seasonal(t)) }
rate_life <- function(t) { k_life / life_mean_days(T_seasonal(t)) }


## 3) Construct the PopJSON model

In [ ]:
popjson <- list(
  model = list(
    title = "Larva with competing Erlang clocks",
    type = "Population",
    url  = "https://github.com/kerguler/Population",
    deterministic = TRUE,
    parameters = list(algorithm="Population", istep=0.01)
  ),
  environ = list(list(id="temp", name="Temperature (C)", url="")),
  functions = list(
    briere1 = ["define", ["T","B","E","a"], ["?",
      ["<=","T","B"], "365.0",
      ["?",[">=","T","E"],"365.0",
        ["min","365.0",
          ["max","1.0",
            ["/","1.0",
              ["exp", ["+","a",["log","T"],["log",["-","T","B"]],["*","0.5",["log",["-","E","T"]]]]]
            ]
          ]
        ]
      ]
    ]]
  ),
  intermediates = list(
    list(id="d2m", value=["briere1", ["temp","TIME_1"], 10, 35, -15]),
    list(id="l2m", value=["briere1", ["temp","TIME_1"], 8, 38, -14]),
    list(id="d2s", value=["*","d2m", 1.0/(k_dev^0.5)]),
    list(id="l2s", value=["*","l2m", 1.0/(k_life^0.5)])
  ),
  populations = list(list(
    id="larva",
    name="Larva stage",
    processes=list(
      list(id="larva_mort", name="Larva lifetime", arbiter="ACC_ERLANG", value=["l2m","l2s"]),
      list(id="larva_dev", name="Larva development", arbiter="ACC_ERLANG", value=["d2m","d2s"])
    ),
    init=list(list(0,0,200))
  )),
  transformations=list(
    list(id="larva_death", name="Larvae dying today", value="larva_mort"),
    list(id="larva_to_pupa", name="Larvae developing to pupa", value="larva_dev")
  )
)
write(toJSON(popjson, pretty=TRUE, auto_unbox=TRUE), "larva_erlang_temp_popjson.json")


## 4) ODE analogue: phase-type representation

In [ ]:
make_rhs <- function(kD, kL) {
  function(t,y,parms) {
    n_grid <- kD*kL
    x <- y[1:n_grid]; P <- y[n_grid+1]; D <- y[n_grid+2]
    lamD <- rate_dev(t); lamL <- rate_life(t)
    dx <- rep(0,n_grid); dP <- 0; dD <- 0
    idx <- function(i,j) (i-1)*kL+j
    for(i in 1:kD) for(j in 1:kL) {
      id <- idx(i,j)
      out_dev <- lamD*x[id]; out_life <- lamL*x[id]
      dx[id] <- dx[id] - (out_dev+out_life)
      if(i<kD) dx[idx(i+1,j)] <- dx[idx(i+1,j)] + out_dev else dP <- dP+out_dev
      if(j<kL) dx[idx(i,j+1)] <- dx[idx(i,j+1)] + out_life else dD <- dD+out_life
    }
    list(c(dx,dP,dD))
  }
}
kD<-k_dev; kL<-k_life; rhs<-make_rhs(kD,kL)
y0 <- c(rep(0,kD*kL),0,0); y0[1]<-200
times<-seq(0,200,by=0.25)
sol <- as.data.frame(ode(y=y0,times=times,func=rhs,parms=NULL,method="lsoda"))
n_grid<-kD*kL
sol <- sol %>% mutate(Larvae=rowSums(.[,2:(n_grid+1)])) %>% rename(Pupa=.[[n_grid+2]],Dead=.[[n_grid+3]])


## 5) Visualization

In [ ]:
df_long <- sol %>% select(time,Larvae,Pupa,Dead) %>% pivot_longer(-time)
ggplot(df_long,aes(x=time,y=value,linetype=name))+geom_line()+
  labs(title="Larval system (ODE analogue)",x="Time (d)",y="Count")+theme_minimal()


In [ ]:
temp_df <- data.frame(time=times,T=T_seasonal(times))
ggplot(temp_df,aes(x=time,y=T))+geom_line()+labs(title="Temperature forcing",x="Time (d)",y="T (C)")+theme_minimal()
